In [ ]:
import pandas as pd

# Load the interested Patients IDs file
interested_ids_df = pd.read_csv('MP_IDs.csv')

# Load the file with IDs and visit details
visit_details_df = pd.read_csv('liste_participants.csv')

# Merge the dataframes based on the ID column
merged_df = pd.merge(interested_ids_df, visit_details_df, on='ID', how='inner')


In [ ]:
def summarize_participation(row):
    visits = []
    if row['V0 pro'] >= 0.5:
        visits.append('0')
    if row['V1 pro'] >= 0.5:
        visits.append('1')
    if row['V2 pro'] >= 0.5:
        visits.append('2')
    if row['V3 pro'] >= 0.5:
        visits.append('3')
    if row['V4 pro'] >= 0.5:
        visits.append('4')
    return ', '.join(visits) if visits else 'None'


In [ ]:
# Extract visit participation columns
merged_df['Visits'] = merged_df.apply(summarize_participation, axis=1)

# Select relevant columns to display
result_df = merged_df[['ID', 'Visits']]

# Print the resulting dataframe
print(result_df)
result_df.to_csv('output.csv', index=False)

In [ ]:
def parse_visits(visits):
    if pd.isna(visits):  # Check for NaN values
        return []
    if isinstance(visits, float):  # Handle unexpected float values
        return []
    if isinstance(visits, int):  # Handle single integer values
        return [visits]
    if isinstance(visits, str):  # Handle string values
        return [int(visit.strip()) for visit in visits.split(',')]
    return []  # Return an empty list for any other unexpected type


In [ ]:
# Initialize an empty dataframe to collect all results
for index, row in ids_and_visits_df.iterrows():
    id_to_search = row['ID']
    visits_to_keep =  parse_visits(row['Visits']) # Convert the string list to an actual list
    
    # Filter the main data dataframe for the specific ID
    filtered_df = main_data_df[main_data_df['ID'] == id_to_search]
    
    # Ensure visits_to_keep is a list of integers
    temp_filtered_list = []
    
    # Keep only the specified rows for this ID using iloc (subtract 1 from visit numbers for zero-based indexing)
    for visit in visits_to_keep:
        if 0 <= visit-1 < len(filtered_df):
            temp_row = filtered_df.iloc[visit-1].copy()
            temp_row['visits'] = visit  # Add the visit number to the row
            temp_filtered_list.append(temp_row)
    
    # Convert the list of rows to a dataframe
    temp_filtered_df = pd.DataFrame(temp_filtered_list)
    
    # Append the result to the all_filtered_data_df
    all_filtered_data_df = pd.concat([all_filtered_data_df, temp_filtered_df], ignore_index=True)

# Reset index for the final dataframe
all_filtered_data_df.reset_index(drop=True, inplace=True)

# Print the resulting dataframe
print(all_filtered_data_df)

# Optionally, save the result to a new CSV file
all_filtered_data_df.to_csv('subject_visits2.csv', index=False)
